## Extracting player height and weight 
**25-08-22**
* weight not available for all players
* height not availabel for all players but a very small minority

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from bs4 import BeautifulSoup
import requests
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", 200)

In [ ]:
# import player list
# players = pd.read_csv('02_database/cbb_player.csv')

In [ ]:
# create empty DFs to store various ways that height/weight is stored
# df_height_5col = pd.DataFrame(columns = ['a','b','c','d','cbb_player_id'])
# df_height_4col = pd.DataFrame(columns = ['a','b','c','cbb_player_id'])
# df_height_3col = pd.DataFrame(columns = ['a','b','cbb_player_id'])
# df_height_error = pd.DataFrame(columns = ['ccb_player_id'])

In [ ]:
# extracting height/weight from player profile
# some players (very small fraction) will not have these details. Usually this is when the player has not played very much, just a few games
# player_ids = players.player_id.tolist()
# for num,player_id in enumerate(player_ids):
#     try:
#         url = 'https://www.sports-reference.com/cbb/players/' + player_id + '.html'
#         page = requests.get(url)
#         soup = BeautifulSoup(page.text,'lxml') 
#         profile = soup.find('div',{'id':'meta'})
#         if profile.find_all('p')[1]:
#             row = [i.text.replace('\xa0','').replace(' ','') for i in profile.find_all('p')[1]]
#             row.append(player_id)
#             if len(row) == 5:
#                 df_height_5col.loc[len(df_height_5col)] = row
#             elif len(row) == 4:
#                 df_height_4col.loc[len(df_height_4col)] = row
#             elif len(row) == 3:
#                 df_height_3col.loc[len(df_height_3col)] = row
#             print(str(num)+' of '+str(len(player_ids)))
#     except:
#         df_height_error.loc[len(df_height_error)] = player_id
#         pass

In [ ]:
## These players have some extra text, so the height/weight data is in the 2nd element

# df_height_4col_fix = pd.DataFrame(columns = ['a','b','c','d','cbb_player_id'])
# player_ids = df_height_4col.cbb_player_id.tolist()
# for num,player_id in enumerate(player_ids):
#     try:
#         url = 'https://www.sports-reference.com/cbb/players/' + player_id + '.html'
#         page = requests.get(url)
#         soup = BeautifulSoup(page.text,'lxml') 
#         profile = soup.find('div',{'id':'meta'})
#         if profile.find_all('p')[2]:
#             row = [i.text.replace('\xa0','').replace(' ','') for i in profile.find_all('p')[2]]
#             row.append(player_id)
#             df_height_4col_fix.loc[len(df_height_4col_fix)] = row
#             print(str(num)+' of '+str(len(player_ids)))
#     except:
#         df_height_error.loc[len(df_height_error)] = player_id
#         pass

In [1633]:
# back up copies
# df_height_5col.to_csv('01_raw_csv_files/cbb_height_weight/df_height_5col.csv',index = False)
# # df_height_4col.to_csv('01_raw_csv_files/cbb_height_weight/df_height_4col.csv',index = False) # can use the _fix instead
# df_height_4col_fix.to_csv('01_raw_csv_files/cbb_height_weight/df_height_4col_fix.csv',index = False)
# df_height_3col.to_csv('01_raw_csv_files/cbb_height_weight/df_height_3col.csv',index = False)
# df_height_error.to_csv('01_raw_csv_files/cbb_height_weight/df_height_error.csv',index = False)

#### Cleaning up height tables
* output of web scraping above falls into 3 categories
    1. 5 column output - most commont type, has height/weight in cm/kg and feet/lb and one empty column
    1a. 4 column output - was originally 4 columns but I fixed the code and it will product 5 columns, so can ignore this
    2. 3 column output - may not have height for everyone, for those who do have height they don't have weight. From this list I just filtered on those who had the height
    3. error - those who did not really have any height info. Usually it would be when a player plays very few games and is very low profile. Just ignore those
* remember that this list needs to **left-join** to the main list since you don't have every player

In [66]:
df_height_5col = pd.read_csv('01_raw_csv_files/cbb_height_weight/df_height_5col.csv')
df_height_4col_fix = pd.read_csv('01_raw_csv_files/cbb_height_weight/df_height_4col_fix.csv')
df_height_3col = pd.read_csv('01_raw_csv_files/cbb_height_weight/df_height_3col.csv')
df_height_error = pd.read_csv('01_raw_csv_files/cbb_height_weight/df_height_error.csv')

In [71]:
# cleaning height_cm and weight_kg [df_height_5col,df_height_4col_fix]
# not going to clean lb and feet, same info
x = pd.concat([df_height_5col,df_height_4col_fix], axis = 0).d\
.str.replace(')','', regex = False)\
.str.replace('(','',regex = False)\
.str.replace('cm','',regex = False)\
.str.replace('kg','',regex = False)\
.str.split(',',expand = True)\
.rename(columns = {0:'height_cm',1:'weight_kg'})
height_weight_temp_1 = pd.concat([pd.concat([df_height_5col.cbb_player_id,df_height_4col_fix.cbb_player_id], axis = 0),x],axis = 1)

# cleaning up df_height_3col
x = df_height_3col[~df_height_3col.b.str.contains('kg')].b\
.str.replace("(",'',regex = False)\
.str.replace(')','',regex=False)\
.str.replace('cm','',regex = False)
height_weight_temp_2 = pd.concat([df_height_3col[~df_height_3col.b.str.contains('kg')].cbb_player_id,x,pd.Series(np.nan)],axis = 1)\
.rename(columns = {'b':'height_cm',0:'weight_kg'})

player_height_weight = pd.concat([height_weight_temp_1,height_weight_temp_2],axis = 0)
player_height_weight.head(2)

,cbb_player_id,height_cm,weight_kg
0,harrison-hawkins-1,183,77
1,parker-wentz-1,175,77


In [75]:
# save as csv
player_height_weight.to_csv('02_database/cbb_player_height_weight.csv',index = False)